In [29]:
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE

def retiro_simbolo(dataframe, list_columns, simbol, simbol2):
    '''
    Esta funcion toma una lista de columnas y reemplaza un simbolo por uno dado por el usuario
    '''
    for columna in list_columns:
        dataframe[columna] = dataframe[columna].str.replace(simbol, simbol2)
        dataframe[columna] = dataframe[columna].str.replace(',','.')
    return dataframe

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [2]:
import mlflow
from mlflow.models import infer_signature
mlflow.set_tracking_uri("http://Mlflow:5000")

In [3]:
import os
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://Minio:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'supersecret'

In [27]:
df = pd.read_csv('diabetes.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')

### procesamiento

df = df.rename(columns=lambda x: x.replace(' ', '_').lower())
df = df.loc[:,['age', 'discharge_disposition_id', 'time_in_hospital',
               'num_lab_procedures', 'num_procedures', 'number_inpatient', 'diag_1',
               'diag_2', 'diag_3', 'number_diagnoses', 'readmitted']]


# df['gender'] = df['gender'].replace({'Female':1,
#                                    'Male':0,
#                                    'Unknown/Invalid' : np.nan})

df['age'] = df['age'].replace({'[0-10)':5,
                               '[10-20)':15, 
                               '[20-30)':25, 
                               '[30-40)':35,
                               '[40-50)':45, 
                               '[50-60)':55,
                               '[60-70)':65, 
                               '[70-80)':75,
                               '[80-90)':85, 
                               '[90-100)':95})

# df['race'] = df['race'].replace({'Caucasian':0,
#                                'AfricanAmerican':1,
#                                'Other':2, 
#                                'Asian':3, 
#                                'Hispanic':4,
#                                '?': np.nan})

df['readmitted'] = df['readmitted'].replace({'NO':0,
                                            '>30':1,
                                            '<30':0})


# df['weight'] = df['weight'].replace({'[0-25)':12.5,
#                                     '[25-50)': 37.5,
#                                     '[50-75)': 62.5,
#                                     '[75-100)': 87.5,
#                                     '[100-125)': 112.5,
#                                     '[125-150)': 137.5, 
#                                     '[150-175)': 162.5, 
#                                     '[175-200)': 187.5, 
#                                     '>200': 200,
#                                     '?': np.nan})

# df['change'] = df['change'].replace({'Ch':1,
#                                     'No':0})

# df['diabetesMed'] = df['diabetesMed'].replace({'Yes':1,
#                                                'No':0})
 
# similar_columns = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 
#                    'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 
#                    'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'insulin', 
#                    'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 
#                    'metformin-rosiglitazone', 'metformin-pioglitazone']

# for col in similar_columns:
#     df[col] = df[col].replace({'No':-1,
#                                'Steady':0,
#                                'Up':1,
#                                'Down':-1})


# df['A1Cresult'] = df['A1Cresult'].replace({'>7': 1,
#                                            '>8': 1,
#                                            'Norm': 0,
#                                            'None': -1})

# df['max_glu_serum'] = df['max_glu_serum'].replace({'>200': 1,
#                                                   '>300': 1,
#                                                   'Norm': 0,
#                                                   'None': -1})   

df = retiro_simbolo(df, ['diag_1', 'diag_2', 'diag_3'], '?', '0')

diag_columns = ['diag_1', 'diag_2', 'diag_3']

for col in diag_columns:
    df.loc[df[col].str.contains('V|E', na=False), col] = 0
    df[col] = df[col].astype(float)
    df[col] = df[col] // 100


# df = df.drop(['max_glu_serum', 'A1Cresult', 'weight'], axis=1)  # tienen mas del 84% de los datos nulos
# df = df.drop(['num_medications', 'diabetesMed', 'change'], axis=1)  # correlaciones +40 y 35
# df = df.drop(['payer_code', 'medical_specialty', 'examide', 'citoglipton'], axis=1)  # no num
# df = df.drop(['encounter_id'], axis=1)  # id

# df = df.drop(['patient_nbr'], axis=1)  # ni idea

df = df.dropna()
df = df.drop_duplicates()

### fin procesamiento

df.head()

,age,discharge_disposition_id,time_in_hospital,num_lab_procedures,num_procedures,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,readmitted
0,5,25,1,41,0,0,2.0,0.0,0.0,1,0
1,15,1,3,59,0,0,2.0,2.0,2.0,9,1
2,25,1,2,11,5,1,6.0,2.0,0.0,6,0
3,35,1,2,44,1,0,0.0,2.0,4.0,7,0
4,45,1,1,51,0,0,1.0,1.0,2.0,5,0


In [30]:
# Separar las características (X) y la variable objetivo (y)
X = df.drop(['readmitted'], axis=1)
y = df['readmitted']

## Escalar las características
scaler = MinMaxScaler()
scaler2 = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = scaler2.fit_transform(X_scaled)

## Balanceo de clases
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

## Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [31]:
EXPERIMENT_NAME = "Readmitted-Survived-Classifier-Experiment"
mlflow.set_experiment(EXPERIMENT_NAME)

current_experiment=dict(mlflow.get_experiment_by_name(EXPERIMENT_NAME))
experiment_id=current_experiment['experiment_id']

2024/05/06 02:22:35 INFO mlflow.tracking.fluent: Experiment with name 'Readmitted-Survived-Classifier-Experiment' does not exist. Creating a new experiment.


In [32]:
model_name = 'Decision Tree'
RUN_NAME = f'Readmitted Classifier Experiment {model_name}'
params = {'max_depth':3, 'min_samples_split':2}
with mlflow.start_run(experiment_id=experiment_id, run_name=RUN_NAME):
    
    model = DecisionTreeClassifier(**params)
    
    model.fit(X_train, y_train)  # Train model
    predictions = model.predict(X_test)  # Predictions

    # Calculate metrics
    accuracy = accuracy_score(y_test, predictions)
    f1 = f1_score(y_test, predictions, average='weighted')  
    
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric(f"{model_name}_accuracy", accuracy)
    mlflow.log_metric(f"{model_name}_f1", f1)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", f"{model_name} model for Readmitted")

    # Infer the model signature
    signature = infer_signature(X_train, model.predict(X_train))
    
    #log the model

    model_info = mlflow.sklearn.log_model(
    sk_model=model,
    artifact_path=f"readmitted_{model_name}_model",
    signature=signature,
    input_example=X_train,
    registered_model_name=f"tracking-readmitted-{model_name}",)

    mlflow.end_run() 

Successfully registered model 'tracking-readmitted-Decision Tree'.
2024/05/06 02:22:58 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-readmitted-Decision Tree, version 1
Created version '1' of model 'tracking-readmitted-Decision Tree'.


In [ ]:
from mlflow import MlflowClient

client = MlflowClient()

client.set_registered_model_tag("tracking-readmitted-Decision Tree", "task", "classification")

In [ ]:
model_name = 'RF'
RUN_NAME = f'Readmitted Classifier Experiment {model_name}'
params = {'n_estimators':200}
with mlflow.start_run(experiment_id=experiment_id, run_name=RUN_NAME):
    
    model = RandomForestClassifier(**params)
    
    model.fit(X_train, y_train)  # Train model
    predictions = model.predict(X_test)  # Predictions

    # Calculate metrics
    accuracy = accuracy_score(y_test, predictions)
    f1 = f1_score(y_test, predictions, average='weighted')  
    
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric(f"{model_name}_accuracy", accuracy)
    mlflow.log_metric(f"{model_name}_f1", f1)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", f"{model_name} model for Readmitted")

    # Infer the model signature
    signature = infer_signature(X_train, model.predict(X_train))
    
    #log the model

    model_info = mlflow.sklearn.log_model(
    sk_model=model,
    artifact_path=f"readmitted_{model_name}_model",
    signature=signature,
    input_example=X_train,
    registered_model_name=f"tracking-readmitted-{model_name}",)

    mlflow.end_run() 

In [33]:
from mlflow import MlflowClient

client = MlflowClient()

client.set_registered_model_tag("tracking-readmitted-RF", "task", "classification")

# load

In [34]:
model_name = "tracking-readmitted-Decision Tree"
model_version = 1

lr = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

2024/05/06 02:22:58 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


In [35]:
lr.predict(X_test)

array([0, 1, 0, ..., 1, 1, 1])

In [36]:
user_input = [-0.060217, -0.516283, 0.201260, -0.257637,	-0.786889, 0.280606, 0.309819, -0.958752, 0.277905, 0.810384]
columns = ['age', 'discharge_disposition_id', 'time_in_hospital', 'num_lab_procedures', 'num_procedures', 
           'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses']

df_pred = pd.DataFrame([user_input], columns=columns)
out_model = lr.predict(df_pred)
out_model

array([1])

In [ ]:
model_name = "tracking-readmitted-RF"
model_version = 1

rf = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

In [ ]:
rf.predict(X_test)

In [ ]:
user_input = [-0.060217, -0.516283, 0.201260, -0.257637,	-0.786889, 0.280606, 0.309819, -0.958752, 0.277905, 0.810384]
columns = ['age', 'discharge_disposition_id', 'time_in_hospital', 'num_lab_procedures', 'num_procedures', 
           'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses']

df_pred = pd.DataFrame([user_input], columns=columns)
out_model = rf.predict(df_pred)
out_model

In [37]:
print('ok_')

ok_
